In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time

def get_treasure_links():
    base_url = "https://www.museum.go.kr/site/main/relic/treasure/list"
    all_data = []
    
    # 페이지 수는 실제 상황에 맞게 수정 (예: 37페이지)
    total_pages = 37
    
    for page in range(1, total_pages + 1):
        start_count = 12 * (page - 1)
        params = {"startCount": start_count}
        
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"페이지 {page} 접속 에러: status {response.status_code}")
            continue
        
        soup = BeautifulSoup(response.text, "html.parser")
        
        # li.card 안의 a.img-box를 찾아서 href를 추출
        cards = soup.select("li.card a.img-box")
        
        if not cards:
            print(f"페이지 {page}에 카드가 없습니다. 구조가 바뀌었을 수 있습니다.")
            continue
        
        for card in cards:
            detail_link = card.get("href", "")
            
            # 원하는 JSON 구조에 맞게 저장
            all_data.append({
                "page": page,
                "link": detail_link  # 예: "/site/main/relic/treasure/view?relicId=358"
            })
            print(f"페이지: {page}, 링크: {detail_link}")

        #break    
        time.sleep(0.5)  # 서버 부하 방지용 임시 지연
    
    return all_data


data = get_treasure_links()
print(f"수집 완료! 총 {len(data)}개의 링크를 저장했습니다.")

페이지: 1, 링크: /site/main/relic/treasure/view?relicId=348
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=349
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=350
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=355
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=358
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=372
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=482
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=487
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=499
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=517
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=518
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=521
수집 완료! 총 12개의 링크를 저장했습니다.


In [3]:
data

[{'page': 1, 'link': '/site/main/relic/treasure/view?relicId=348'},
 {'page': 1, 'link': '/site/main/relic/treasure/view?relicId=349'},
 {'page': 1, 'link': '/site/main/relic/treasure/view?relicId=350'},
 {'page': 1, 'link': '/site/main/relic/treasure/view?relicId=355'},
 {'page': 1, 'link': '/site/main/relic/treasure/view?relicId=358'},
 {'page': 1, 'link': '/site/main/relic/treasure/view?relicId=372'},
 {'page': 1, 'link': '/site/main/relic/treasure/view?relicId=482'},
 {'page': 1, 'link': '/site/main/relic/treasure/view?relicId=487'},
 {'page': 1, 'link': '/site/main/relic/treasure/view?relicId=499'},
 {'page': 1, 'link': '/site/main/relic/treasure/view?relicId=517'},
 {'page': 1, 'link': '/site/main/relic/treasure/view?relicId=518'},
 {'page': 1, 'link': '/site/main/relic/treasure/view?relicId=521'}]

In [2]:
#data
base_url = "https://www.museum.go.kr"
url_list = [base_url + item["link"] for item in data]
url_list = url_list[:1]
url_list

['https://www.museum.go.kr/site/main/relic/treasure/view?relicId=348']

In [5]:
import os
import re
import json
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs

def parse_museum_page(url):
    """
    주어진 URL(국립중앙박물관 유물 상세)에서 
    - 대표(큰) 이미지
    - 썸네일 이미지 목록
    - 유물 정보(이름/국적/시대/재질/분류 등)
    - 본문 content
    등을 추출하여 하나의 JSON(dict)으로 반환
    """

    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"페이지 응답 에러(HTTP {response.status_code}): {url}")
    soup = BeautifulSoup(response.text, "html.parser")

    print(response.text)

    # 결과를 담을 딕셔너리
    result = {
        "url": url,          # 어떤 URL을 파싱했는지
        "img": None,         # 대표 이미지
        "thumbs": [],        # 썸네일 목록
        "info": {},          # 상세 정보(제목 등)
        "content": ""        # 본문
    }

    # [썸네일 추출]
    thumb_slides = soup.select("div.swiper-container.gallery-thumbs a.swiper-slide")
    for tslide in thumb_slides:
        style_text = tslide.get("style", "")
        # style="background:url('/relic_image/xxx.jpg') ..."
        matches = re.findall(r'url\(["\'](.+?)["\']\)', style_text)
        if matches:
            result["thumbs"].append(matches[0])

    # [대표 이미지: 썸네일 중 첫 번째 사용]
    if result["thumbs"]:
        result["img"] = result["thumbs"][0]

    # [상세정보 파싱]  : outview-tit-box > outview
    outview_box = soup.select_one("div.outview-tit-box div.outview")
    if outview_box:
        # 제목
        title_el = outview_box.select_one("strong.outveiw-tit")
        if title_el:
            result["info"]["title"] = title_el.get_text(strip=True)

        # ul.outview-list
        info_list = outview_box.select("ul.outview-list > li")
        for li in info_list:
            strong_el = li.select_one("strong")
            p_el = li.select_one("p")
            if strong_el and p_el:
                key_text = strong_el.get_text(strip=True)
                val_text = p_el.get_text(strip=True)
                result["info"][key_text] = val_text

    # [본문 내용 파싱] : view-info-cont2 > p
    content_el = soup.select_one("div.view-info-cont.view-info-cont2 > p")
    if content_el:
        result["content"] = content_el.get_text(strip=True)

    return result

def get_relic_id_from_url(url: str) -> str:
    """
    URL의 쿼리스트링(querystring)에서 relicId=XXX 값을 추출.
    예: https://www.museum.go.kr/...relicId=36559443 -> '36559443'
    """
    parsed = urlparse(url)
    qs = parse_qs(parsed.query)
    # relicId 값이 존재하면 첫 번째 사용, 없으면 "unknown"
    relic_id = qs.get("relicId", ["unknown"])[0]
    return relic_id

def download_image(img_url: str, folder_path: str):
    """
    img_url을 다운로드해 folder_path 내에 저장.
    img_url이 절대경로(http...)가 아니면 
    https://www.museum.go.kr + img_url 로 가정.
    """
    if not img_url:
        return  # 값이 없으면 스킵

    if img_url.startswith("http"):
        full_url = img_url
    else:
        # 국립중앙박물관 사이트이므로 도메인 붙이기
        full_url = "https://www.museum.go.kr" + img_url

    try:
        r = requests.get(full_url, timeout=10)
        if r.status_code == 200:
            # url의 마지막 파일명 추출
            filename = os.path.basename(img_url)
            save_path = os.path.join(folder_path, filename)
            with open(save_path, "wb") as f:
                f.write(r.content)
            print(f" [이미지] 저장 완료: {save_path}")
        else:
            print(f" [이미지] 응답 에러: {full_url}, status={r.status_code}")
    except Exception as e:
        print(f" [이미지] 다운로드 예외: {full_url}, err={e}")


base_dir = "relics"

for url in url_list:
    # relicId 추출
    relic_id = get_relic_id_from_url(url)
    print(f"\n=== {relic_id} 처리 중 ===")

    # 2) parse 함수로 데이터 추출
    data = parse_museum_page(url)

    # 3) relics/<relic_id> 폴더 생성
    relic_folder = os.path.join(base_dir, relic_id)
    if not os.path.exists(relic_folder):
        os.makedirs(relic_folder)

    # 4) json 파일로 저장
    json_path = os.path.join(relic_folder, "info.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f" [JSON] 저장 완료: {json_path}")

    # 5) 이미지 다운로드 (대표 이미지 + 썸네일)
    # 대표 이미지
    download_image(data["img"], relic_folder)
    # 썸네일 목록
    for thumb_url in data["thumbs"]:
        download_image(thumb_url, relic_folder)        


=== 348 처리 중 ===


























<!Doctype html>
<html lang="ko">
<head prefix="og: http://ogp.me/ns#">

<!-- Google tag (gtag.js) [S] -->
<script async src="https://www.googletagmanager.com/gtag/js?id=G-WDJ8QWXHST"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'G-WDJ8QWXHST');
</script>
<!-- Google tag (gtag.js) [E] -->

<meta charset="utf-8" />
<meta name="format-detection" content="telephone=no" />


<meta http-equiv="X-UA-Compatible" content="IE=edge" />
<!-- 20190621: 접근성 수정  s-->

<meta name="viewport"
	content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1, user-scalable=yes, target-densitydpi=medium-dpi" />



<title>
		
		
		
		
		
		

		
		
		
	
	
		
		감산사 석조미륵보살입상 | 국보·보물 검색:국립중앙박물관
	
	</title>
<!-- 20190621: 접근성 수정  s-->
<!-- SEO -->
<meta property="og:type" content="website">
<meta property="og:site_name" content="국립중앙박물관">
<meta pr